In [1]:
import pandas as pd
import os
import re
import concurrent.futures
from openai import OpenAI, BadRequestError, APIConnectionError, RateLimitError
from tqdm import tqdm
import time

# ================= 配置区域 =================

# 1. 大模型配置 (阿里云 DashScope)
LLM_API_KEY = "sk-fa13f585000140deabdfa506b25a7f3d"
LLM_API_BASE = "https://dashscope.aliyuncs.com/compatible-mode/v1"
MODEL_NAME = "qwen3-30b-a3b-instruct-2507"

# 2. 并发数量
MAX_WORKERS = 5

# 3. 文件路径
try:
    CURRENT_DIR = os.path.dirname(os.path.abspath(__file__))
except NameError:
    CURRENT_DIR = os.getcwd()

# 输入文件 (使用你刚刚上传的文件)
INPUT_FILENAME = "api_retrieval_split_result_v2.xlsx" 
INPUT_PATH = os.path.join(CURRENT_DIR, INPUT_FILENAME)

# 输出文件 (生成一个新文件)
OUTPUT_PATH = os.path.join(CURRENT_DIR, "final_analysis_comparison_aliyun_v2.xlsx")

# ===========================================

# 初始化客户端
client = OpenAI(api_key=LLM_API_KEY, base_url=LLM_API_BASE)

def clean_model_output(text):
    """清洗模型输出"""
    if not text: return ""
    text = re.sub(r'<think>.*?</think>', '', text, flags=re.DOTALL)
    text = text.replace('```', '').strip()
    return text

def merge_columns(row, prefix, count):
    """合并多个召回结果列"""
    content_list = []
    for i in range(1, count + 1):
        col_name = f"{prefix}{i}"
        if col_name not in row and prefix == "召回结果":
             col_name = f"召回结果{i}"

        if col_name in row and pd.notna(row[col_name]):
            val = str(row[col_name]).strip()
            # 简单去重和去空
            if val and val not in content_list:
                content_list.append(f"[{i}]: {val}")
    
    # 默认截取 2500 字
    return "\n".join(content_list)[:2500]

def call_llm_smart(prompt, retries=3):
    """
    智能调用函数：包含重试 + 合规拦截自动降级机制
    """
    current_prompt = prompt
    
    for i in range(retries):
        try:
            chat_response = client.chat.completions.create(
                model=MODEL_NAME,
                messages=[
                    {"role": "system", "content": "你是一个精准的数据分析师。请直接输出分析结果，严禁包含思考过程(<think>)，严禁输出客套话。字数严格控制在100字以内。"},
                    {"role": "user", "content": current_prompt}
                ],
                temperature=0.1,
                max_tokens=256
            )
            return clean_model_output(chat_response.choices[0].message.content)
            
        except BadRequestError as e:
            # 捕获 400 错误 (包含 data_inspection_failed)
            err_body = e.body or {}
            err_code = err_body.get('code', '')
            
            # 如果是合规检测失败 (data_inspection_failed)
            if 'data_inspection' in str(e) or 'inappropriate content' in str(e):
                print(f"⚠️ 触发合规拦截，尝试缩减文本重试 ({i+1}/{retries})...")
                # 【关键策略】: 将 Prompt 长度砍半，试图丢弃包含敏感词的部分
                if len(current_prompt) > 500:
                    current_prompt = current_prompt[:len(current_prompt)//2] + "\n...(后半部分因风控截断)"
                    continue # 使用缩短后的 prompt 立即重试
                else:
                    return "内容触发平台风控，无法评估"
            else:
                return f"请求参数错误: {str(e)[:50]}"
                
        except (APIConnectionError, RateLimitError) as e:
            # 网络或限流错误，等待后重试
            time.sleep(2)
            continue
            
        except Exception as e:
            return f"未知错误: {str(e)[:50]}"
            
    return "调用超时或多次失败"

def analyze_single_row(row):
    """
    处理单行数据
    """
    try:
        # 如果之前已经有成功的结果，跳过（方便断点续传）
        # 这里假设你要重新跑，所以不做跳过检查
        
        question = str(row.get('问题', row.get('原问题', '')))
        
        baseline_content = merge_columns(row, "召回结果", 5)
        scheme1_content = merge_columns(row, "方案1_召回结果", 3)
        scheme2_content = merge_columns(row, "方案2_召回结果", 3)

        if not baseline_content: baseline_content = "（无召回内容）"
        if not scheme1_content: scheme1_content = "（无召回内容）"
        if not scheme2_content: scheme2_content = "（无召回内容）"

        # 构造 Prompts
        prompts = [
            # 1. 方案1 评估
            f"【问题】：{question}\n【检索内容】：{scheme1_content}\n请一句话评估：上述检索内容是否包含回答问题所需的关键事实？缺失了什么？(限100字)",
            # 2. 方案2 评估
            f"【问题】：{question}\n【检索内容】：{scheme2_content}\n请一句话评估：上述检索内容是否包含回答问题所需的关键事实？缺失了什么？(限100字)",
            # 3. 方案1 vs 原召回
            f"【问题】：{question}\n【原检索】：{baseline_content}\n【新检索(长难句扩写)】：{scheme1_content}\n请对比：新检索相比原检索，准确性是提升还是下降？主要体现在哪？(限100字，直接给结论)",
            # 4. 方案2 vs 原召回
            f"【问题】：{question}\n【原检索】：{baseline_content}\n【新检索(潜在追问)】：{scheme2_content}\n请对比：新检索相比原检索，信息丰富度是提升还是下降？是否引入了噪音？(限100字，直接给结论)"
        ]

        # 串行执行4次调用 (单行内串行，行与行之间并行)
        results = []
        for p in prompts:
            res = call_llm_smart(p)
            results.append(res)

        return results

    except Exception as e:
        err_msg = f"处理异常: {str(e)}"
        return [err_msg, err_msg, err_msg, err_msg]

def main():
    print(f"正在读取文件: {INPUT_PATH}")
    if not os.path.exists(INPUT_PATH):
        print("❌ 文件不存在！")
        return

    df = pd.read_excel(INPUT_PATH)
    total_rows = len(df)
    print(f"共 {total_rows} 条数据，使用 {MAX_WORKERS} 线程并发处理 (含智能容错)...")

    results_list = []

    with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        rows = df.to_dict('records')
        results_iterator = list(tqdm(executor.map(analyze_single_row, rows), total=total_rows, desc="LLM智能分析中"))
        results_list = results_iterator

    # 解包结果
    col1, col2, col3, col4 = [], [], [], []
    for res in results_list:
        col1.append(res[0])
        col2.append(res[1])
        col3.append(res[2])
        col4.append(res[3])

    df["方案1_效果评估"] = col1
    df["方案2_效果评估"] = col2
    df["方案1_vs_原召回_对比"] = col3
    df["方案2_vs_原召回_对比"] = col4

    print("分析完成，正在保存...")
    try:
        df.to_excel(OUTPUT_PATH, index=False)
        print(f"🎉 处理成功！结果保存在: {OUTPUT_PATH}")
    except PermissionError:
        print("❌ 保存失败：请关闭 Excel 文件后重试。")

if __name__ == "__main__":
    main()

正在读取文件: d:\jupyter\api_retrieval_split_result_v2.xlsx
共 83 条数据，使用 5 线程并发处理 (含智能容错)...


LLM智能分析中:  36%|███▌      | 30/83 [00:24<00:29,  1.79it/s]

⚠️ 触发合规拦截，尝试缩减文本重试 (1/3)...


LLM智能分析中:  75%|███████▍  | 62/83 [00:50<00:13,  1.54it/s]

⚠️ 触发合规拦截，尝试缩减文本重试 (1/3)...


...

⚠️ 触发合规拦截，尝试缩减文本重试 (1/3)...


LLM智能分析中:  84%|████████▍ | 70/83 [00:57<00:08,  1.50it/s]

⚠️ 触发合规拦截，尝试缩减文本重试 (1/3)...


LLM智能分析中: 100%|██████████| 83/83 [01:08<00:00,  1.21it/s]

分析完成，正在保存...
🎉 处理成功！结果保存在: d:\jupyter\final_analysis_comparison_aliyun_v2.xlsx
